# 1. Import Libraries and Data Files

In [1]:
# Import libraries

import os
import sys
import random
import math
import numpy as np
import skimage.io
import skimage.draw
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd
from multiprocessing import freeze_support
from google.cloud import storage
from IPython.display import Image

# Import mask rcnn code framework
from mrcnn import model as modellib, utils
from mrcnn.config import Config


In [1]:
# Transfer training and validation data files from Cloud Bucket to local storage in VM instance 
# DON'T NEED TO RUN THIS MORE THAN ONCE!

# client = storage.Client()
# DATA_BUCKET = 'w207-final-project'
# databucket = client.get_bucket(DATA_BUCKET)
# datafiles = databucket.list_blobs()

# Transfer Training Files
# files=[a.name for a in datafiles if a.name.startswith('train_clean')]

# Transfer Validation Files
# files=[a.name for a in datafiles if a.name.startswith('val_clean')]

# Transfer Validation Masks
# files=[a.name for a in datafiles if a.name.endswith('png')]

# print(files)
# Copy files from bucket into local folder
# for file in files:
#     blob = databucket.get_blob(file)
#     blob.download_to_filename(file)




In [3]:
# Check if all files are copied over
list = os.listdir('train_clean') # dir is your directory path
number_files = len(list)
print("# train image files:", number_files)
list = os.listdir('val_clean') # dir is your directory path
number_files = len(list)
print("val image files:", number_files)
list = os.listdir('val_clean_masks') # dir is your directory path
number_files = len(list)
print("val image mask files:", number_files)

# train image files: 3220
val image files: 359
val image mask files: 357


# 2. Set Up Model Training 


## a. Set file directory paths

In [4]:
ROOT_DIR = os.path.abspath("")

# Path to trained weights file

# Original weights path (run this during first iteration)
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# New weights path (provide updated path to weights from generated models here)
WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_object_0100.h5")

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

## b. Define classes for model configuration and loading images for training/validation

In [5]:

class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We are using the NVIDIA T4 Tensore Core GPU, which has 16GB memory.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + drivable area

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 2

    # Number of validation steps to run at the end of every training epoch.
    # A bigger number improves accuracy of validation stats, but slows
    # down the training.
    VALIDATION_STEPS = 10
    
    # Backgone architecture supported values are: resnet50, resnet101.
    # You can also provide a callable that should have the signature
    # of model.resnet_graph. If you do so, you need to supply a callable
    # to COMPUTE_BACKBONE_SHAPE as well
    BACKBONE = "resnet101" # Default
    # BACKBONE = "resnet50"

    # Skip detections below % confidence specified
    # DETECTION_MIN_CONFIDENCE = 0.5
    DETECTION_MIN_CONFIDENCE = 0.9

In [6]:
# Load training and validation images
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        """Load a subset of the driving dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """

        # Add classes. We have only one class to add.
        self.add_class("object", 1, "drivable area")
        
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset + "_clean")

        # Set labels to appropriate image file names (training or validation)
        if subset == 'train':
            labels = train_labels_cleaned
        else:
            labels = val_labels_cleaned
        
        # Given an image, get all the items that are considered drivable area
        # For each item, save the x and y vertices into two lists, x and y
        for index, image in labels.iterrows():

            polygons = []
            objects = []
            for obj in image['labels']:
                x = []
                y = []
                if obj['category'] == 'drivable area':
                    for coord in obj['poly2d'][0]['vertices']:
                        coord = [round(num) for num in coord]
                        # Set max value of coordinates (max x is 1280, max y is 720)
                        if coord[0] >= 1280:
                            x.append(1279)
                        else:
                            x.append(coord[0])
                        if coord[1] >= 720:
                            y.append(719)
                        else:
                            y.append(coord[1])
                    polygons.append({'name': 'polygon',
                                      'all_points_x': x,
                                      'all_points_y': y})
                    objects.append('drivable area')
                    
            image_path = os.path.join(dataset_dir, image['name'])

            # Set dictionary of names and ID for different object classes other than background
            name_dict = {"drivable area": 1}
            
            num_ids = [name_dict[a] for a in objects]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=image['name'],  # use file name as a unique image id
                path=image_path,
                width=1280, height=720,
                polygons=polygons,
                num_ids=num_ids
                )
        return

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """

        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [8]:

def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom(ROOT_DIR, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom(ROOT_DIR, "val")
    dataset_val.prepare()

    print("Training network heads")

    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=40, # Tune this hyperparameter
                layers='heads')


## c. Load object labels

In [9]:
# Load JSON files for image segment labels

with open('train_clean/bdd100k_labels_images_train.json') as json_file:
    train_labels = json.load(json_file)

with open('val_clean/bdd100k_labels_images_val.json') as json_file:
    val_labels = json.load(json_file)

In [10]:

# REQUIRED: Normalize semi-structured JSON data into a pd dataframe
# Note that train_labels json file contains both labels for training and validation set 
train_labels_normalized = pd.json_normalize(train_labels)

train_filenames = []
val_filenames = []

# REMEMBER TO REPLACE AFTER INITIAL TESTING
for myfile in os.listdir(ROOT_DIR + "/train_clean"):
    if myfile.endswith(".jpg"):
        train_filenames.append(os.path.join(myfile))

for myfile in os.listdir(ROOT_DIR + "/val_clean"):
    if myfile.endswith(".jpg"):
        val_filenames.append(os.path.join(myfile))

# Save labels for only the images within the folder contents
train_labels_cleaned = train_labels_normalized.loc[train_labels_normalized['name'].isin(train_filenames)]

# val_labels_split
val_labels_cleaned = train_labels_normalized.loc[train_labels_normalized['name'].isin(val_filenames)]



In [11]:
# TEST ONLY: Verify image sizes and dimensions

# from PIL import Image as im

# image_folder = 'train_clean/'

# for filename in train_filenames:

#     with im.open(image_folder + filename) as image:
#         # convert image to numpy array
#         # plt.imshow(image)
#         image_data = np.asarray(image)
#     if image_data.shape != (720, 1280, 3):
#         print("Image Shape for " + filename + " is " + image_data)

# image_folder = 'val_clean/'

# for filename in val_filenames:

#     with im.open(image_folder + filename) as image:
#         # convert image to numpy array
#         # plt.imshow(image)
#         image_data = np.asarray(image)
#     if image_data.shape != (720, 1280, 3):
#         print("Image Shape for " + filename + " is " + image_data)


In [14]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpus
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
    print("error")

1 Physical GPUs, 1 Logical GPU


In [15]:

config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

# weights_path = COCO_WEIGHTS_PATH # For first run only
weights_path = WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])


In [16]:
train(model)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/jupyter/logs/object20220412T1819/mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
2022-04-12 18:19:33.857004: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 18:19:33.857480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 18:19:33.857799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-12 18:19:33.858420: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative 

Epoch 1/40


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_

1/2 [==============>...............] - ETA: 17s - batch: 0.0000e+00 - size: 1.0000 - loss: 29.9578 - rpn_class_loss: 21.8579 - rpn_bbox_loss: 8.0989 - mrcnn_class_loss: 0.0010 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00

2022-04-12 18:20:48.062987: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-04-12 18:20:48.063032: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-04-12 18:20:48.063087: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-04-12 18:20:48.064073: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-04-12 18:20:49.083697: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-04-12 18:20:49.084316: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-04-12 18:20:49.154177: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer

2/2 [==============================] - ETA: 0s - batch: 0.5000 - size: 1.0000 - loss: 23.7647 - rpn_class_loss: 16.4438 - rpn_bbox_loss: 7.3203 - mrcnn_class_loss: 5.2014e-04 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 

2022-04-12 18:20:49.332143: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /home/jupyter/logs/object20220412T1819/plugins/profile/2022_04_12_18_20_49/w207-final-project-2.trace.json.gz
2022-04-12 18:20:49.434986: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /home/jupyter/logs/object20220412T1819/plugins/profile/2022_04_12_18_20_49

2022-04-12 18:20:49.444072: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to /home/jupyter/logs/object20220412T1819/plugins/profile/2022_04_12_18_20_49/w207-final-project-2.memory_profile.json.gz
2022-04-12 18:20:49.447775: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: /home/jupyter/logs/object20220412T1819/plugins/profile/2022_04_12_18_20_49
Dumped tool data for xplane.pb to /home/jupyter/logs/object20220412T1819/plugins/profile/2022_04_12_18_20_49/w207-final-project-

2/2 [==============================] - 62s 44s/step - batch: 0.5000 - size: 1.0000 - loss: 23.7647 - rpn_class_loss: 16.4438 - rpn_bbox_loss: 7.3203 - mrcnn_class_loss: 5.2014e-04 - mrcnn_bbox_loss: 0.0000e+00 - mrcnn_mask_loss: 0.0000e+00 - val_loss: 5.1008 - val_rpn_class_loss: 2.1336 - val_rpn_bbox_loss: 2.9672 - val_mrcnn_class_loss: 2.3842e-07 - val_mrcnn_bbox_loss: 0.0000e+00 - val_mrcnn_mask_loss: 0.0000e+00
Epoch 2/40
2/2 [==============================] - 8s 7s/step - batch: 0.5000 - size: 1.0000 - loss: 7.0294 - rpn_class_loss: 1.2848 - rpn_bbox_loss: 3.3471 - mrcnn_class_loss: 0.2656 - mrcnn_bbox_loss: 1.6785 - mrcnn_mask_loss: 0.4533 - val_loss: 7.6435 - val_rpn_class_loss: 0.2933 - val_rpn_bbox_loss: 5.2483 - val_mrcnn_class_loss: 0.0505 - val_mrcnn_bbox_loss: 1.1178 - val_mrcnn_mask_loss: 0.9336
Epoch 3/40
2/2 [==============================] - 8s 8s/step - batch: 0.5000 - size: 1.0000 - loss: 5.2803 - rpn_class_loss: 0.1597 - rpn_bbox_loss: 4.4664 - mrcnn_class_loss: 0.0

In [25]:
print(tf.__version__)

2.8.0
